#  theano scan usage

In [1]:
import numpy as np
import theano
import theano.tensor as tt

## map a function over a list 

In [2]:
s_x = tt.ivector()
s_y, _ = theano.scan(
    fn = lambda x:x*x,
    sequences = [s_x])
fn = theano.function([s_x], s_y)
fn([1,2,3,4,5]) #[1,4,9,16,25]

array([ 1,  4,  9, 16, 25], dtype=int32)

## zip a function over a list  

In [3]:
s_x1 = tt.ivector()
s_x2 = tt.ivector()
s_y, _ = theano.scan(
    fn = lambda x1,x2:x1**x2,
    sequences = [s_x1, s_x2])
fn = theano.function([s_x1,s_x2], s_y)
fn([1,2,3,4,5],[0,1,2,3,4]) #[1,2,9,64,625]


array([  1,   2,   9,  64, 625], dtype=int32)

## accumulate a list  

In [4]:
s_x = tt.ivector()
v_sum = theano.shared(np.int32(1))
s_y, update_sum = theano.scan(
    lambda x,y:x+y,
    sequences = [s_x],
    outputs_info = [v_sum]) # initial value
fn = theano.function([s_x], s_y, updates=update_sum)

print( v_sum.get_value() )# 0
print( fn([1,2,3,4,5]) ) # [1,3,6,10,15]
print( v_sum.get_value() )# 15
print( fn([-1,-2,-3,-4,-5]) )# [14,12,9,5,0]
print( v_sum.get_value() ) # 0

1
[ 2  4  7 11 16]
1
[  0  -2  -5  -9 -14]
1


## orbit of logistic map 

In [5]:
s_x = tt.fscalar()
s_lambda = tt.fscalar()
s_t = tt.iscalar()
s_y, _ = theano.scan(
    fn = lambda x,l: l*x*(1-x),
    outputs_info = [s_x],
    non_sequences = [s_lambda],
    n_steps = s_t
)
fn = theano.function([s_x, s_lambda, s_t], s_y, allow_input_downcast=True)

print( fn(.75, 4., 10) ) #a stable orbit

print( fn(.65, 4., 10) ) #a chaotic orbit

[0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75]
[0.91       0.3275999  0.8811129  0.41901192 0.97376364 0.10219204
 0.3669953  0.929239   0.2630156  0.77535355]


## Taps - Fibonacci 

In [6]:
'''
don't know why this function is not correctly worked.
'''
s_x0 = tt.iscalar()
s_x1 = tt.iscalar()
s_n = tt.iscalar()
s_y, _ = theano.scan(
    fn = lambda x1,x2: x1+x2,
    outputs_info = [{'initial':[s_x0, s_x1], 'taps':[-2,-1]}],
    n_steps = s_n
)
fn_fib = theano.function([s_x0, s_x1, s_n], s_y)
fn_fib(1,1,10)

AttributeError: 'list' object has no attribute 'ndim'

## scan indexing  

In [7]:
x = tt.dvector('x')
def step(idx, array):
    return array[idx] + idx
results, updates = theano.scan(fn=step,
                               sequences=tt.arange(x.shape[0]),
                               non_sequences=[x])
f = theano.function([x], results)
f([1., 0., 2.13])

array([1.  , 1.  , 4.13])

##  graph scan

In [12]:
a = tt.vector("a")
b = a + a ** 10  
f = theano.function([a], b)   
f([1])
theano.printing.pydotprint(b, outfile="../pic/pic1.png", var_with_name_simple=True)  
theano.printing.pydotprint(f, outfile="../pic/pic2.png", var_with_name_simple=True)

The output file is available at ../pic/pic1.png
The output file is available at ../pic/pic2.png


In [27]:
a = tt.vector("a")
b = a
for i in range(10):
    b = b + a 
f = theano.function([a],b)
theano.printing.pydotprint(b, outfile="../pic/scan1.png", var_with_name_simple=True)  
theano.printing.pydotprint(f, outfile="../pic/scan2.png", var_with_name_simple=True)

The output file is available at ../pic/scan1.png
The output file is available at ../pic/scan2.png


In [28]:
a = tt.vector("a")
b = tt.vector("b")
s_y, _ = theano.scan(
    fn = lambda x,y: x+y,
    outputs_info = [a],
    non_sequences = [a],
    n_steps = 10
)
res = s_y[-1]
fn = theano.function([a], res, allow_input_downcast=True)
fn([3])
theano.printing.pydotprint(res, outfile="../pic/scan3.png", var_with_name_simple=True)  
theano.printing.pydotprint(fn, outfile="../pic/scan4.png", var_with_name_simple=True)


The output file is available at ../pic/scan3.png
The output file is available at ../pic/scan4.png


# make theano function and get gradient
- https://groups.google.com/forum/#!topic/theano-users/Z-vXqB-2SL8

In [ ]:
'''
can calculate function value, but can not use for autograd 
'''
def func(x1,x2):
    a = tt.exp(2*x1+2*x2)
    return(a)

x1 = tt.dscalar()
x2 = tt.dscalar()
y_ = func(x1,x2)
fn = theano.function([x1,x2],y_)
fn(1,-1)

In [ ]:
import autograd.numpy as np 
from autograd import grad

In [ ]:
def func(x1,x2):
    a = np.exp(2*x1+2*x2)
    return(a)

x1 = tt.dscalar()
x2 = tt.dscalar()
y_ = func(x1,x2)
fn = theano.function([x1,x2],y_)
fn(1,-1)

In [ ]:
gradFunc = grad(func)
x1 = np.float64(1)
x2 = np.float64(-1)
gradFunc(x1,x2)

## pymc3 custom function, check np or tt 

In [ ]:
import pymc3 as pm
import autograd.numpy as np 
from autograd import grad
import statsmodels.api as sm

In [11]:
'''
if np is used , TypeError: sum() got an unexpected keyword argument 'out'
are returned. Please be sure to use tt, and check function is 
correctly set by overlapping common function
'''
EPS = np.finfo(float).eps
def logiFunc(x):
    v = 1./(1. + np.exp(-x) )
    v = np.clip(v,EPS,1-EPS)
    return(v)
 
def pred(X,w):
    linear = np.sum(w*X,axis=1)
    y =  logiFunc(linear)  
    return(y)

def logLikelihood(t,X,w):
    y = pred(X,w) 
    ll = t * np.log(y) + (1-t)*np.log(1-y) 
    ll = np.sum(ll)
    return(ll)



In [12]:
# data preparation 
df = sm.datasets.anes96.load_pandas().data
exog = ["TVnews","selfLR","PID","educ"]
endog = "vote"

# set data 
m = len(exog)
x = df[exog].values
y = df[endog].values
x_shared = theano.shared(x)
# sampling parameters 

nIter = 500
with pm.Model() as model2:
    coeff = pm.Normal('beta', mu=0, sigma=100,shape=m)
    pm.DensityDist('likelihood',logLikelihood , observed=dict(t=y,X=x,w=coeff))

    # fit the model
    trace2 = pm.sample(draws=nIter)

NameError: name 'sm' is not defined